# Example selectors

만약 많은 수의 예시를 가지고 있다면, 프롬프트에 포함할 예시를 선택해야 할 수도 있다. 예시 선택기(Example Selector)는 이를 담당하는 클래스다.

기본 인터페이스는 아래와 같이 정의된다:

```python
class BaseExampleSelector(ABC):
    """Interface for selecting examples to include in prompts."""

    @abstractmethod
    def select_examples(self, input_variables: Dict[str, str]) -> List[dict]:
        """Select which examples to use based on the inputs."""
        
    @abstractmethod
    def add_example(self, example: Dict[str, str]) -> Any:
        """Add new example to store."""
```


정의해야 할 유일한 메소드는 `select_examples` 메소드다. 이 메소드는 입력 변수를 받아들이고 예시의 목록을 반환한다. 그 예시들이 어떻게 선택되는지는 각 구현에 따라 달라진다.

## Examples

예시 선택기를 사용하기 위해서는 예시의 목록을 생성해야 한다. 이들은 일반적으로 예시 입력과 출력이어야 한다. 데모 목적으로, 우리가 영어를 이탈리아어로 번역하는 예시를 선택한다고 상상해 보자.

In [36]:
examples = [
    {"input": "hi", "output": "ciao"},
    {"input": "bye", "output": "arrivaderci"},
    {"input": "soccer", "output": "calcio"},
]

## Custom Example Selector

단어의 길이를 기반으로 어떤 예시를 선택할지 결정하는 예시 선택기를 작성해 보자

In [37]:
from langchain_core.example_selectors.base import BaseExampleSelector


class CustomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        # 이는 입력의 일부가 'text' 키가 될 것이라는 지식을 가정한다.
        new_word = input_variables["input"]
        new_word_length = len(new_word)

        # 최적의 매치와 그 길이 차이를 저장할 변수를 초기화한다.
        best_match = None
        smallest_diff = float("inf")

        # 각 예시를 순회한다.
        for example in self.examples:
            # 예시의 첫 번째 단어와의 길이 차이를 계산한다.
            current_diff = abs(len(example["input"]) - new_word_length)

            # 현재의 것이 길이에 더 가까우면 최적의 매치를 업데이트한다.
            if current_diff < smallest_diff:
                smallest_diff = current_diff
                best_match = example

        return [best_match]

In [38]:
example_selector = CustomExampleSelector(examples)

In [39]:
example_selector.select_examples({"input": "okay"})

[{'input': 'bye', 'output': 'arrivaderci'}]

In [40]:
example_selector.add_example({"input": "hand", "output": "mano"})

In [41]:
example_selector.select_examples({"input": "okay"})

[{'input': 'hand', 'output': 'mano'}]

## Use in a Prompt

이제 이 예시 선택기를 프롬프트에서 사용할 수 있습니다.

In [42]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate.from_template("Input: {input} -> Output: {output}")

In [43]:
prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    suffix="Input: {input} -> Output:",
    prefix="Translate the following words from English to Italain:",
    input_variables=["input"],
)

print(prompt.format(input="word"))

Translate the following words from English to Italain:

Input: hand -> Output: mano

Input: word -> Output:
